#  Machine translate App




---



## Setup necessary files

In [10]:
! pip install spacy -q
! pip install torchtext==0.6.0 -q
! pip install pyvi -q
! pip install dill -q
! pip install gdown -q

import nltk
nltk.download('wordnet')

!npm install -g localtunnel -q
!pip install streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.8 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



changed 22 packages, and audited 23 packages in 752ms

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


## Buid and Run web

In [2]:
import gdown

file_ids = {
    'TRG.Field': '1P8eFuMSdubhbrDJGorqfH5KLM2eQpxnF',
    'SRC.Field': '1ESxk93bOEWoWfBFcGH0sD7So415G8LQd',
    'TransformerModel.py': '1efL-9Kv-qq4O-Lnilys54YvqrVmu2tHg',
    'transformer.pth': '1Aqww-iJtPzkEaJN_m9EYxqMoo-qr2Q-c',
}

for file_name in list(file_ids):
  url = f"https://drive.google.com/uc?id={file_ids[file_name]}"
  gdown.download(url, file_name, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1P8eFuMSdubhbrDJGorqfH5KLM2eQpxnF
To: /content/TRG.Field
100%|██████████| 17.1M/17.1M [00:00<00:00, 127MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1ESxk93bOEWoWfBFcGH0sD7So415G8LQd
To: /content/SRC.Field
100%|██████████| 17.8M/17.8M [00:00<00:00, 47.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1efL-9Kv-qq4O-Lnilys54YvqrVmu2tHg
From (redirected): https://drive.google.com/uc?id=1efL-9Kv-qq4O-Lnilys54YvqrVmu2tHg&confirm=t&uuid=ccbe866c-215e-4cdb-80ff-93469e666efc
To: /content/TransformerModel.py
100%|██████████| 60.8k/60.8k [00:00<00:00, 55.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Aqww-iJtPzkEaJN_m9EYxqMoo-qr2Q-c
From (redirected): https://drive.google.com/uc?id=1Aqww-iJtPzkEaJN_m9EYxqMoo-qr2Q-c&confirm=t&uuid=dffead34-2905-4276-a712-3143eaba0253
To: /content/transformer.pth
100%|██████████| 348M/348M [00:06<00:00, 50.7MB/s]


In [3]:
%%writefile app.py
from PIL import Image
import streamlit as st
from TransformerModel import *
import pandas as pd
import dill as pickle

opt = {
    'src_lang': 'en_core_web_sm',
    'trg_lang': 'en_core_web_sm',
    'max_strlen': 160,
    'batchsize': 1500,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'd_model': 512,
    'n_layers': 6,
    'heads': 8,
    'dropout': 0.1,
    'lr': 0.0001,
    'epochs': 15,
    'printevery': 200,
    'k': 5,
}

# Introduction
st.title("Translate from English to VietNamese")

@st.cache_resource
def Read_filed():
  with open(f"SRC.Field","rb") as f:
     SRC = pickle.load(f)
  with open(f"TRG.Field","rb") as f:
     TRG = pickle.load(f)
  return SRC, TRG

@st.cache_resource
def load_model():
    model = Transformer(len(SRC.vocab), len(TRG.vocab), opt['d_model'], opt['n_layers'], opt['heads'], opt['dropout'])

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    model.load_state_dict(torch.load(f'transformer.pth', map_location=torch.device(opt['device'])))
    model = model.to(opt['device'])
    return model

with st.spinner('Wait for callling 🤖...'):
    SRC, TRG = Read_filed()
    model = load_model()

st.success('🤖 is ready !')

text = st.text_area("Input some text 💬:", "")
output_text = st.empty()
output_text.text_area("Vietnamese Output Text")
btn = st.button("Translate")
result = ""
if btn:
    textsplit = text.splitlines()
    for x in textsplit:
      trans_sent = translate_sentence(x, model, SRC, TRG, opt['device'], opt['k'], opt['max_strlen'])
      result += trans_sent
      result = result +'\n'
    output_text.text_area("Vietnamese Output Text", result)

    st.success('Done !', icon="✅")

Writing app.py


In [11]:
!streamlit run app.py --server.port 8501 &>/content/logs.txt & curl ipv4.icanhazip.com
!lt --port 8501

34.74.7.88
your url is: https://tall-swans-raise.loca.lt
^C
